#### Imports & Constants

In [116]:
import sys
import pathlib
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

sys.path.append(pathlib.Path("..").resolve().as_posix())

from src.coder import TargetCoder
from src.io_utils import write_stdout_to_file, write_df_to_csv

scaler = MinMaxScaler()
le = LabelEncoder()

# 110 columns in the dataset
# 160k+ rows
pd.set_option('display.max_columns', 110)
pd.set_option('display.max_rows', 1000)

DATASET_PATH = pathlib.Path("..") / "data" / "raw" / "players.csv"
MAIN_COLUMNS = [7, 14, 15, 31, 32, 33, 35, 41, 42, 43, 44, 45, 46]
DETAILED_COLUMNS =  list(range(47, 76))
USED_COLUMNS = MAIN_COLUMNS + DETAILED_COLUMNS

COLS_TO_NORMALIZE = range(0, 38)

print(USED_COLUMNS)

[7, 14, 15, 31, 32, 33, 35, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


#### Load Data 

In [117]:
df = pd.read_csv(DATASET_PATH, usecols=USED_COLUMNS)

##### Droppings

In [118]:
def drop_goalkeepers(df: pd.DataFrame) -> None:
    """
    Drops all goalkeepers from the dataset inplace
    """
    df.drop(df[df["player_positions"].str.contains("GK")].index, inplace=True)
    
def drop_mentality_composure(df: pd.DataFrame) -> None:
    """
    Drops all columns related to mentality composure
    Reason:
        - The column has 20k+ Nans
    """
    df.drop(columns="mentality_composure", inplace=True)

### Proccess positions

In [119]:
def process_positions(df: pd.DataFrame) -> None:
    """
    Processes the player_positions column inplace
    Effect:
        - Maps the positions to the main positions (first position in the string)
        - Move the column to the end of the dataframe
        - Renames the column to position to "position"
    """
    # map positions
    df["player_positions"] = df["player_positions"].str.split(",").str[0].map(TargetCoder.map)
    # move the column to the end
    df["position"] = df.pop("player_positions")
    
def encode_positions(df: pd.DataFrame) -> pd.DataFrame:
    """
    Encodes the position column inplace
    """
    df = df.copy()
    df["position"] = df["position"].map(TargetCoder.encode)
    return df

##### One hot encode work rate & preferred_foot

In [120]:
# RUN ONCE BECAUSE THE WORKRATE COLUMNS WOULD HAVE BEEN DROPED
def get_encoded_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Processes the work_rate column inplace using one hot encoding
    Effect:
        - One hot encodes the work_rate
        - One hot encodes the preferred_foot
    """
    new_df = pd.get_dummies(df, columns=['work_rate', 'preferred_foot'])
    return new_df


##### Normalize Columns

In [121]:
def normalize_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalizes the dataframe inplace
    Effect:
        - Normalizes the dataframe inplace
    """
    normalizeed_df = df.copy()
    normalizeed_df.iloc[:, COLS_TO_NORMALIZE] = scaler.fit_transform(df.iloc[:, COLS_TO_NORMALIZE])
    return normalizeed_df

#### Process Data

In [122]:
""" 
KEEP ORDER
"""

# drop columns
drop_goalkeepers(df)
drop_mentality_composure(df)
# map positions
process_positions(df)

# encode & normalize data 
encoded_df = get_encoded_df(df)
encoded_df = encode_positions(encoded_df)
encoded_df = normalize_df(encoded_df)

# df.head()
encoded_df.head()

,height_cm,weight_kg,weak_foot,skill_moves,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,position,work_rate_High/High,work_rate_High/Low,work_rate_High/Medium,work_rate_Low/High,work_rate_Low/Low,work_rate_Low/Medium,work_rate_Medium/High,work_rate_Medium/Low,work_rate_Medium/Medium,preferred_foot_Left,preferred_foot_Right
0,0.288462,0.295082,0.50,0.75,0.947368,0.9375,0.904110,1.000000,0.168831,0.553846,0.879518,0.988235,0.710843,0.920000,0.903614,0.988235,0.939759,0.941176,0.7625,1.000,0.987013,0.907895,0.973333,0.972222,0.9750,0.811765,0.688312,0.740260,0.518987,0.927711,0.441860,0.144578,0.952941,0.929412,0.764706,0.178571,0.130952,0.117647,0,0,0,0,0,0,0,0,1,0,1,0
1,0.596154,0.508197,0.75,1.00,0.947368,0.9875,0.835616,0.932432,0.233766,0.800000,0.867470,1.000000,0.891566,0.826667,0.927711,0.952941,0.927711,0.811765,0.7125,0.950,0.922078,0.960526,0.960000,0.916667,0.5750,0.976471,0.961039,0.896104,0.759494,0.987952,0.616279,0.168675,0.941176,0.823529,0.870588,0.142857,0.250000,0.152941,1,0,1,0,0,0,0,0,0,0,0,1
2,0.500000,0.508197,0.25,0.75,0.947368,0.9000,0.863014,0.945946,0.233766,0.569231,0.831325,0.882353,0.457831,0.880000,0.915663,0.952941,0.891566,0.858824,0.7625,0.925,0.948052,0.947368,0.960000,0.902778,0.9250,0.882353,0.532468,0.753247,0.582278,0.951807,0.430233,0.349398,0.917647,0.858824,0.811765,0.226190,0.190476,0.188235,2,0,1,0,0,0,0,0,0,0,1,0
3,0.788462,0.754098,0.75,0.75,0.723684,0.9625,0.835616,0.864865,0.259740,0.907692,0.783133,0.952941,0.771084,0.853333,0.987952,0.894118,0.831325,0.823529,0.7625,0.925,0.701299,0.736842,0.866667,0.847222,0.3000,0.964706,0.675325,0.753247,0.936709,0.927711,0.860465,0.120482,0.882353,0.847059,0.941176,0.178571,0.369048,0.200000,0,0,0,0,0,0,0,0,1,0,0,1
5,0.307692,0.262295,0.75,0.75,0.710526,0.7250,0.945205,0.932432,0.584416,0.553846,0.891566,0.741176,0.506024,0.973333,0.771084,0.941176,0.831325,0.705882,0.9250,0.975,0.727273,0.710526,0.826667,0.916667,0.8625,0.635294,0.441558,0.753247,0.506329,0.771084,0.558140,0.698795,0.894118,0.964706,0.705882,0.559524,0.559524,0.541176,3,0,0,1,0,0,0,0,0,0,0,1


#### Save Data

In [123]:

cleaned_data_path = pathlib.Path("..") / "data" / "processed" / "cleaned_data.csv"
cleaned_data_info_path = pathlib.Path("..") / "data" / "processed" / "cleaned_data_info.txt"

processed_data_path = pathlib.Path("..") / "data" / "processed" / "processed_data.csv"
processed_data_info_path = pathlib.Path("..") / "data" / "processed" / "processed_data_info.txt"

def create_cleaned_data_files(df: pd.DataFrame) -> None:
    write_df_to_csv(cleaned_data_path, df)
    write_stdout_to_file(cleaned_data_info_path, lambda: df.info(verbose=True, show_counts=True))
    
def create_processed_data_files(df: pd.DataFrame) -> None:
    write_df_to_csv(processed_data_path, df)
    write_stdout_to_file(processed_data_info_path, lambda: df.info(verbose=True, show_counts=True))

# create cleaned data files
create_cleaned_data_files(df)
# create processed data files
create_processed_data_files(encoded_df)